In [4]:
""" If you do not have these libraries on your computer/Jupyter, you can install them using the lines below (uncomment them). """

"""
import sys
!{sys.executable} -m pip install customtkinter
!{sys.executable} -m pip install --upgrade transformers torch torchvision torchaudio
!{sys.executable} -m pip install customtkinter
!{sys.executable} -m pip install sentencepiece
"""

In [5]:
""" If you do not have these libraries on your computer/Jupyter, you can install them using the lines below (uncomment them). """

"""
!pip install PyMuPDF
!pip install transformers
!pip install torch
!pip install sumy
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install PyMuPDF transformers torch customtkinter
!pip install tf-keras
!pip install transformers PyPDF2 tqdm
!pip install --upgrade jupyter ipywidgets
!jupyter nbextension enable --py widgetsnbextension
"""

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 2.1 MB/s eta 0:00:06
     ---- ----------------------------------- 1.6/12.8 MB 3.6 MB/s eta 0:00:04
     --------- ------------------------------ 2.9/12.8 MB 4.3 MB/s eta 0:00:03
     ------------ --------------------------- 3.9/12.8 MB 4.7 MB/s eta 0:00:02
     ------------ --------------------------- 3.9/12.8 MB 4.7 MB/s eta 0:00:02
     ------------------ --------------------- 6.0/12.8 MB 4.7 MB/s eta 0:00:02
     ---------------------- ----------------- 7.3/12.8 MB 4.8 MB/s eta 0:00:02
     -------------------------- ------------- 8.4/12.8 MB 4.9 MB/s eta 0:00:01
     ------------------------------ --------- 9.7/12.8 MB 5.0 MB/s eta 0:00:01
     ---------------------------------- ----- 11.0/12.8 MB 5.1 MB/s eta 0

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [6]:
# Importing necessary libraries

import customtkinter as ctk
from tkinter import filedialog, messagebox
from tkinter.scrolledtext import ScrolledText
import fitz
from transformers import pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration
from tqdm.notebook import tqdm

In [7]:
# Function to split text into word-based chunks

def chunk_text(text, max_words=300):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i+max_words])
        chunks.append(chunk)
    return chunks

In [8]:
# Loading the model and tokenizer
# Load the pretrained T5 model and tokenizer for summarization

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
# Creating the summarization pipeline

def chunk_text(text, max_words=300):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunk = " ".join(words[i:i+max_words])
        chunks.append(chunk)
    return chunks

In [10]:
# This might take a few seconds

summarizer = pipeline("summarization", model="t5-small")

Device set to use cpu


In [11]:
# Extracting text from a PDF file

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [12]:
# Selecting PDF, summarizing, and displaying in GUI

def select_and_summarize_pdf():
    file_path = filedialog.askopenfilename(filetypes=[("PDF Files", "*.pdf")])
    if not file_path:
        return
    try:
        full_text = extract_text_from_pdf(file_path)

        chunk_word_size = 250
        text_chunks = chunk_text(full_text, max_words=chunk_word_size)
        all_summaries = []
        final_summary = ""
        for i, chunk in enumerate(tqdm(text_chunks, desc="Parçalar özetleniyor")):
            # Mümkünse modelin maksimum token uzunluğuna göre girdiyi sınırlamak daha iyidir.
            # Basitlik için, parçalamak için hala kelime sayısını kullanıyoruz.
            # Pipeline tokenizasyon ve kırpmayı halledecektir.
            summary = summarizer(
                chunk,
                max_length=150,  # Her bir parçanın özetinin maksimum uzunluğu
                min_length=30,   # Her bir parçanın özetinin minimum uzunluğu
                do_sample=False
            )
            all_summaries.append(summary[0]['summary_text'])

            # Tüm bireysel özetleri birleştir
            final_summary = " ".join(all_summaries)
    
            
            result_box.insert("end", summary[0]['summary_text'] + "\n\n")
            # GUI'nin güncellenmesini zorla
            app.update_idletasks() # Bu satır GUI'nin anlık olarak güncellenmesini sağlar

        save_button.configure(state="normal")

    except Exception as e:
        messagebox.showerror("Hata", f"Bir hata oluştu:\n{str(e)}")

In [13]:
# Saving the summary to a file

def save_summary():
    summary = result_box.get("1.0", "end").strip()
    if summary:
        file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text File", "*.txt")])
        if file_path:
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(summary)
            messagebox.showinfo("Success", "Summary saved successfully!")

In [14]:
# Building and launching the GUI interface

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.geometry("700x550")
app.title("PDF Özetleyici - Yapay Zeka Destekli")

select_button = ctk.CTkButton(app, text="📄 PDF Seç", command=select_and_summarize_pdf, font=("Helvetica", 16))
select_button.pack(pady=20)

result_box = ScrolledText(app, wrap="word", font=("Helvetica", 14), bg="#2b2b2b", fg="white", insertbackground="white", height=20)
result_box.pack(padx=20, pady=10, fill="both", expand=True)

save_button = ctk.CTkButton(app, text="💾 Özeti Kaydet", command=save_summary, font=("Helvetica", 16), state="disabled")
save_button.pack(pady=20)

app.mainloop()

Parçalar özetleniyor:   0%|          | 0/36 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (841 > 512). Running this sequence through the model will result in indexing errors
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have bee